# 总体架构

&emsp;&emsp;&emsp;&emsp;早期的机器阅读理解模型大多基于检索技术，但是信息检索主要依赖关键词的匹配，而在很多情况下，单纯依靠问题和文章片段的文字匹配找到的答案与问题并不相关。随着深度学习的发展，机器阅读理解步入了神经网络时代。本文将介绍基于深度学习的机器阅读理解模型的架构，探究其提升性能的原因。

&emsp;&emsp;&emsp;&emsp;基于深度学习的MRC模型构造各异，但是经过多年的实践和探索，逐渐形成了稳定框架结构。本节将介绍MRC模型共有的总体架构。

&emsp;&emsp;&emsp;&emsp;MRC模型的输入为文章和问题。因此，首先要对这两部分进行数字化编码，将其变成可以被计算机处理的信息单元。在编码的过程中，模型需要保留原有语句在文章中的语义。因此，每个单词、短语和句子的编码必须建立在理解上下文的基础上。我们把模型中进行编码的模块称为编码层。

&emsp;&emsp;&emsp;&emsp;接下来，由于文章和问题之间存在相关性，所以模型需要建立文章和问题之间的联系。例如，如果问题中出现关键词“河流”，而文章中出现关键词“长江”。虽然两个词不完全一样，但是其语义编码接近。因此，文章中“长江”一词以及邻近的语句将成为模型回答问题时的重点关注对象。这可以通过注意力机制加以解决。在这个过程中，MRC模型将文章和问题的语义结合在一起进行考量，进一步加深模型对于两者的理解。我们将这个模块称为交互层。交互层可以让模型聚焦文章和问题的语义联系，借助于文章的语义分析加深对问题的理解，同时借助于问题的语义分析加深对文章的理解。

&emsp;&emsp;&emsp;&emsp;经过交互层，模型建立起文章和问题之间的语义联系，接下来就可以预测问题的答案了。完成预测功能的模块称为输出层。MRC任务的答案有多种类型，因此输出层的具体形式需要和任务的答案类型相关联。此外，输出层确定模型优化时的评估函数和损失函数。

# 编码层

&emsp;&emsp;&emsp;&emsp;与其他基于深度学习的自然语言处理模型类似，MRC模型首先需要将文字形式的文章和问题转化成词向量。编码层一般采用类似的分布式算法对文本进行分词和向量化处理，然后加入字符编码等更丰富的信息，并采用上下文编码获得每个单词在具体语境中的含义。

## 词表的建立与初始化

&emsp;&emsp;&emsp;&emsp;首先，模型对训练文本进行分词以得到其中所有的单词。然后，根据阈值选取出现次数超过一定次数的单词组成词表，词表以外的单词视为非词表词（Out-Of-Vocabulary, OOV），用特殊单词<UNK\>表示。这样，模型可以得到一个大小为|V|的词表，每个词表中的词用一个d维向量表示。接下来，我们有两种方法获得词表中的单词向量：
    <br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;1、保持词表向量不变，即采用预训练词表中的向量（如Word2Vec的300维向量），在训练过程中不进行改变；
    <br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;2、将词表中的向量视为参数，在训练过程中和其他参数一起求导并优化。这里，既可以使用预训练词表向量进行初始化，也可以选择随机初始化。

&emsp;&emsp;&emsp;&emsp;第一种选择的优势是模型参数少，训练初期收敛较快；第二种选择的优势是可以根据实际数据调整词向量的值，以达到更好的训练效果。而采用预训练词表向量初始化一般可以使得模型在优化的最初几轮获得明显比随机初始化方法更优的结果。

&emsp;&emsp;&emsp;&emsp;在编码层中，为了更准确地表示每个单词在语句中的语义，除了词向量外，还经常对命名实体（named entity）和词性（part-of-speech）进行向量化处理。如果命名实体共有N种，则建立大小为N的命名实体表，每种命名实体用一个长度为d_N的向量表示；如果词性共有P种，则建立大小为P的词性表，每种词性用一个长度为d_P的向量表示。两个表中的向量均为可训练的参数。然后，用文本分析包，如spaCy，获得文章和问题中的每个词的命名实体和词性，再将对应向量拼接在词向量之后。由于一个词的命名实体属性与词性和这个词所在的语句有关，因此用这种方式获得向量编码可以更好地表示单词的语义，在许多模型中性能都有明显的提升。

&emsp;&emsp;&emsp;&emsp;另一种在机器阅读理解中非常有效的单词编码是精确匹配（exact matching）编码。exact matching编码适用于文章中的单词。对于文章中的单词w，检查w是否出现在问题中：如果是，w的精确匹配编码为1，否则为0，然后将这个二进制位拼接在单词向量后。精确匹配编码可以使模型快速找到文章中出现了问题单词的部分，而许多问题的答案往往就在这部分内容的附近。

## 字符编码

&emsp;&emsp;&emsp;&emsp;文本处理中时常会出现拼写错误，这时通过字符组合，往往可以识别正确的单词形式。此外，许多语言中存在词根的概念，即词的一部分是一个常见且有固定含义的子词。在单词的理解中，字符和子词具有很强的辅助作用。

&emsp;&emsp;&emsp;&emsp;为了更好地利用字符信息，在编码层中采用字符编码，即每个字符用一个向量表示。但是，由于单词的长度不一，每个单词可能有不同个数的字符向量。为了产生一个固定长度的字符信息向量，我们可以将多个字符向量合并成一个编码。最常用的模型是字符CNN。

&emsp;&emsp;&emsp;&emsp;设一个单词有K个字符，且对应的K个字符向量为（c_1, c_2, ..., c_k），每个向量维度为c。字符CNN利用一个窗口大小为W且有f个输出通道的CNN获得（K-W+1）个f维向量。然后，采用最大池化方法求得这些向量每一个维度上的最大值，形成一个f维向量作为结果。

In [2]:
# 字符卷积神经网络Char-CNN的PyTorch实现

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class CharCNNMaxpooling(nn.Module):
    # char_num为字符表大小，char_dim为字符向量长度，window_size为CNN窗口长度，out_channels为CNN输出通道数
    def __init__(self, char_num, char_dim, window_size, out_channels):
        super(CharCNNMaxpooling, self).__init__()
        # 字符表向量，共有char_num个向量，每个维度为char_dim
        self.char_embed = nn.Embedding(num_embeddings=char_num, embedding_dim=char_dim)
        # 1个输入通道，out_channels个输出通道，过滤器大小为window_size*char_dim
        self.cnn = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(window_size, char_dim))

    # 输入char_ids为batch组文本，每个文本长度为seq_len，每个词含word_len个字符编号（0~char_num-1），输入维度为batch*seq_len*word_len
    # 输出res为所有单词的字符向量表示，维度是batch*seq_len*out_channels
    def forward(self, char_ids):
        # 根据字符编号得到字符向量，结果维度为batch*seq_len*word_len*char_dim
        x = self.char_embed(char_ids)
        # 合并前两维并变成单通道，结果维度（batch*seq_len）*1*word_len*char_dim
        x_unsqueeze = x.view(-1, x.shape[2], x.shape[3]).unsqueeze(1)
        # CNN，结果维度为（batch*seq_len）*out_channels*new_seq_len*1
        x_cnn = self.cnn(x_unsqueeze)
        # 删除最后一维，结果维度为（batch*seq）*out_channels*new_seq_len
        x_cnn_result = x_cnn.squeeze(3)
        # 最大池化，遍历最后一维求最大值，结果维度为（batch*seq_len）*out_channels
        res, _ = x_cnn_result.max(2)
        return res.view(x.shape[0], x.shape[1], 1)